In [ ]:
import csv
import cv2
import numpy as np


IsWindows = False

if IsWindows:
    import ntpath
    originalpath = 'MyOwnData/'
else:
    originalpath = 'data/'

lines = []
with open(originalpath + 'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for line in reader:
        lines.append(line)
        
car_camera_images = []
steering_angles = []

steering_correction = 0.25

for line in lines:
    source_path = line[0]
    if IsWindows:
        drive, path_and_file = ntpath.splitdrive(source_path)
        path, filename = ntpath.split(path_and_file)
    else:
        filename = source_path.split('/')[-1]
        
    left_filename = filename.replace('center', 'left')
    right_filename = filename.replace('center', 'right')
    
    center_path = originalpath + 'IMG/' + filename
    left_path = originalpath + 'IMG/' + left_filename
    right_path = originalpath + 'IMG/' + right_filename
    
    image_center = cv2.imread(center_path)
    image_left = cv2.imread(left_path)
    image_right = cv2.imread(right_path)
    
    steering_angle = float(line[3])
    steering_left = steering_angle + steering_correction
    steering_right = steering_angle - steering_correction
    
    car_camera_images.extend([image_center, image_left, image_right])
    steering_angles.extend([steering_angle, steering_left, steering_right])
    
X_train = np.array(car_camera_images)
y_train = np.array(steering_angles)

print('The shape of the image data is', X_train.shape)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Dropout, Cropping2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D

## Inspired from VGG-16
model = Sequential()
model.add(Lambda(lambda x: (x/255.0) - 0.5, input_shape = (160,320,3)))

model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=4)

model.save('model.h5')